#  CRIMES  IN  BOSTON

# Table of contents


 INTRODUCTION

#  Introduction

Crime rates vary from place to place depending on many factors. Crimes include but not limit to murders, rapes, robberies, assaults, burglaries, thefts, auto thefts, arson, law enforcement employees, police officers and so on. They can be categorized into two types e.g., violent crimes and property crime. A violent crime is a crime in which an offender or perpetrator uses or threatens to use force upon a victim. This entails both crimes in which the violent act is the objective, such as murder or rape, as well as crimes in which violence is the means to an end (Wikipedia). Property crime, on the other hand, is a crime that includes, among other crimes, burglary, larceny, theft, motor vehicle theft, arson, shoplifting, and vandalism. Property crime is a crime to obtain money, property, or some other benefit. This may involve force, or the threat of force, in cases like robbery or extortion (Wikipedia). The crime cost per annual in the United Of America have been estimated from \$690 billion up to \\$3.41 trillion [1]. Understanding the crimes' nature and roots is very crucial for individual, society, and nation as a whole. First of all, it can help individual better prepare themselves not to become the victim. It can also help society to take action to minimize crime rate for the safety and wealth of the community. Moreover, reducing crime rate can help a country minimizing its expense on social safety and increasing the harmony of the people.
In this study, I aim at integrating the crime data in Boston with the Fourquare map data to identify the relatioship between crime types and districts that crimes happened with the venues in the Foursquare. The relationship can reveal what kind of crimes more often happen in a specific area of Boston city.

# Dataset 

Thee datasets are used in this study including i) list of United States cities by crime rate [2], ii) US crime incident report [3], and iii) Foursquare [4]. In this section, we shall explore the data by using visualization techniques to get a quick understanding of the data at hand.
In US crime incident dataset, we select only three features from the whole dataset. They are including city, total violent crime, and total property crime. These three feature can tell us which city is safer comparing to the others. In the Boston crime dataset which is our main objective of the project, we select some features such as OFFENSE_CODE_GROUP, District, Latitude, and Longitude to integrate with Foursquare dataset to work on map later.
Whenever we make a call to Foursquare database, we get a list of name, unique ID, location, and category which can be integrate with crime data that also contain the location. Hence, utilizing these two datasets can help us identify the relaship between crimes and venues.

# Method 

In this analysis, two main methods including visualization and clustering are adopted. Visualization is an effective method to explore data before we apply any data mining or machine learning methods. It also gives us a quick understanding of our data so that we can investigate the interesting problems later.
Moreover, clustering methods aim at grouping the data based on their characteristics. In our problem, clusterig method can be used to cluster the neighborhoods of the districts in Boston from our crime data based on their venues that obtained from Foursquare. Crime data also contain different features which are types and numbers of crimes that happened in those districts.